In [1]:
!pip install scikit-surprise

In [2]:
import pandas as pd
import numpy as np
import surprise
import os 
os.chdir(r"C:\Users\dai\Desktop\S-AI\MachineLearning\Day14\filmtrust")

In [3]:
ratings = pd.read_csv(r"C:\Users\dai\Desktop\S-AI\MachineLearning\Day14\filmtrust\ratings.txt",sep=' ',names = ['uid','iid','rating'])
ratings.head()

,uid,iid,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [4]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


In [83]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))# UID, IID RATING only this format and sequence is understand by surprise.reader()
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

In [77]:
data.raw_ratings

[(1, 1, 2.0, None),
 (1, 2, 4.0, None),
 (1, 3, 3.5, None),
 (1, 4, 3.0, None),
 (1, 5, 4.0, None),
 (1, 6, 3.5, None),
 (1, 7, 3.5, None),
 (1, 8, 3.0, None),
 (1, 9, 2.5, None),
 (1, 10, 4.0, None),
 (1, 11, 4.0, None),
 (1, 12, 4.0, None),
 (2, 13, 2.0, None),
 (3, 14, 0.5, None),
 (3, 15, 1.0, None),
 (3, 16, 4.0, None),
 (3, 17, 3.0, None),
 (3, 18, 4.0, None),
 (3, 19, 4.0, None),
 (3, 20, 2.5, None),
 (3, 21, 1.5, None),
 (3, 22, 2.0, None),
 (3, 23, 3.0, None),
 (3, 24, 4.0, None),
 (3, 25, 1.5, None),
 (3, 26, 1.0, None),
 (3, 27, 4.0, None),
 (3, 28, 3.0, None),
 (3, 29, 3.0, None),
 (3, 30, 4.0, None),
 (3, 31, 3.0, None),
 (3, 32, 2.0, None),
 (3, 33, 2.5, None),
 (3, 34, 3.0, None),
 (3, 35, 4.0, None),
 (3, 36, 2.0, None),
 (3, 37, 2.0, None),
 (3, 38, 2.0, None),
 (3, 39, 4.0, None),
 (3, 40, 4.0, None),
 (3, 41, 4.0, None),
 (3, 42, 0.5, None),
 (3, 43, 4.0, None),
 (3, 44, 4.0, None),
 (3, 45, 2.5, None),
 (3, 46, 1.0, None),
 (3, 47, 2.5, None),
 (3, 48, 4.0, None),
 

In [87]:
similarity_options = {'name': 'cosine', 'user_based': True} # for item based 'False'
# Default k = 40
algo = surprise.KNNBasic(sim_options = similarity_options)
output = algo.fit(data.build_full_trainset()) # Compute missing value and fill ,, it also calculate the expected rating for all

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [79]:
output.

## List of User IDs

In [8]:
ratings['uid'].unique()

array([   1,    2,    3, ..., 1506, 1507, 1508], dtype=int64)

Expected rating for user 100 for item 217:

In [90]:
pred = algo.predict(uid='100',iid='900')# 
print(pred.est) # estimated predicted rating (its a tuple)

3.0028030537791928


In [11]:
pred

Prediction(uid='100', iid='900', r_ui=None, est=3.0028030537791928, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

Total Items:

In [13]:
iids = ratings['iid'].unique()
print(iids)

[   1    2    3 ... 2069 2070 2071]


The list of items rated by user 100:

In [15]:
u_iid = ratings[ratings['uid']==100]['iid'].unique() 
print("List of items rated by user 100:", u_iid)
print("No. of items rated by user {0}: {1}".format(100, len(u_iid)))

List of items rated by user 100: [215]
No. of items rated by user 100: 1


List of the items not rated by user 50:

In [17]:
iids_to_predict = np.setdiff1d(iids, u_iid) # 
print("Items not rated by 100 or those items for which the expected ratings are to be predicted:",iids_to_predict )

Items not rated by 100 or those items for which the expected ratings are to be predicted: [   1    2    3 ... 2069 2070 2071]


In [18]:
len(iids_to_predict)

2070

Extracting the estimated rating for iids_to_predict

In [20]:
testset = [[100,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)
exp_ratings = [ (predictions[i].iid,predictions[i].est) for i in range(0,len(predictions)) ]
exp_ratings = pd.DataFrame(exp_ratings, columns=['iid','est_rating'])
exp_ratings.sort_values(by='est_rating',ascending=False).head()

,iid,est_rating
482,484,4.0
543,545,4.0
1763,1765,4.0
1339,1341,4.0
1764,1766,4.0


Tuning for best K

In [22]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

### User-Based Filtering

In [24]:
param_grid = {'k': np.arange(30,70,10),  'user_based':[True]}
param_grid

{'k': array([30, 40, 50, 60]), 'user_based': [True]}

In [25]:
kfold = KFold(n_splits=5, random_state=23, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid,measures=['rmse', 'mae'], cv=kfold)

In [26]:
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Best Score & Paramter:

In [28]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8641633357915124
{'k': 60, 'user_based': True}


We can now use the algorithm that yields the best rmse:

In [30]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


The recommendations can be generated for any user with the object **algo**.

### Item-Based Filtering

In [33]:
param_grid = {'k': np.arange(30,70,10), 'user_based':[False]}
param_grid

{'k': array([30, 40, 50, 60]), 'user_based': [False]}

In [34]:
kfold = KFold(n_splits=5, random_state=23, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid,measures=['rmse', 'mae'], cv=kfold)

In [35]:
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Best Score:

In [37]:
print(gs.best_score['rmse'])

0.8641633357915124


Best Parameter:

In [39]:
print(gs.best_params['rmse'])

{'k': 60, 'user_based': False}


We can now use the algorithm that yields the best rmse:

In [41]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


The recommendations can be generated for any user with the object **algo**.